In [1]:
import pytest
#Boîte A : Tous les titres contiennent le mot "notification" (différents émetteurs/destinataires)
#Boîte B : Tous les emails venant de l'émetteur blabla@gmail.com (différents titres/destinataires)

#todo: généraliser la liste des features / réfléchir à d'autres cas
#scénarios pour classifier

In [2]:
import pandas as pd
features = pd.read_csv('data/emails.csv', sep = ';')
features.head(5)

,destinaire,emetteur:blabla,notification,output
0,False,False,True,1
1,True,False,True,1
2,False,False,True,1
3,False,True,False,2
4,True,True,False,2


In [3]:
features.describe()

,output
count,18.000000
mean,1.500000
std,0.514496
min,1.000000
25%,1.000000
50%,1.500000
75%,2.000000
max,2.000000


In [4]:
features = pd.get_dummies(features)
features.iloc[:,0:].head(5)

,destinaire,emetteur:blabla,notification,output
0,False,False,True,1
1,True,False,True,1
2,False,False,True,1
3,False,True,False,2
4,True,True,False,2


In [5]:
import numpy as np
#labels corresponds to the output result
labels = np.array(features['output'])

In [6]:
print(labels)

[1 1 1 2 2 2 1 1 1 2 2 2 1 1 1 2 2 2]


In [7]:
features= features.drop('output', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

In [8]:
print(features)

[[False False  True]
 [ True False  True]
 [False False  True]
 [False  True False]
 [ True  True False]
 [ True  True False]
 [False False  True]
 [ True False  True]
 [False False  True]
 [False  True False]
 [ True  True False]
 [ True  True False]
 [False False  True]
 [ True False  True]
 [False False  True]
 [False  True False]
 [ True  True False]
 [ True  True False]]


In [9]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [10]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (13, 3)
Training Labels Shape: (13,)
Testing Features Shape: (5, 3)
Testing Labels Shape: (5,)


In [11]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 60, random_state = 42)


#Use of the model
rf.fit(train_features, train_labels);

In [30]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
print(test_features)
print(predictions)

[[False False  True]
 [ True False  True]
 [False False  True]
 [ True  True False]
 [False  True False]]
[1. 1. 1. 2. 2.]


In [13]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.0 degrees.


In [14]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 100.0 %.


In [15]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
class Email_classifier():
    
    def __init__(self, mail_box):
    # body of the constructor
        self.__mail_box = mail_box
    
    def classify(self, email):
        features = self.prepare_data_learn(email)
        folder_number = self.__rf.predict([features])
        folder_number = round(folder_number[0])
        tree = self.__rf.apply([features])
        print(tree)
        self.print_forest()
        return self.dico_folder_name[folder_number]
    
    def print_forest(self):
        for i in range(len(self.__rf.estimators_)):
            tree = self.__rf.estimators_[i]
            export_graphviz(tree, out_file = 'tree' + str(i) +'.dot', feature_names = ['emetteur: blabla','destinataire', 'notification' ], rounded = True, precision = 1)
            (graph, ) = pydot.graph_from_dot_file('tree' + str(i) +'.dot')
            graph.write_png('tree' + str(i) +'.png')

    def learn(self, data):
        all_email = []
        all_labels = []
        self.dico_folder_name = {}
        folder_number = 1
        for folder_name in data:
            all_email += data[folder_name]
            self.dico_folder_name[folder_number] = folder_name
            all_labels += [folder_number]*len(data[folder_name])
            folder_number += 1
        #print(all_labels)
        #print(all_email)
        features = list(map(self.prepare_data_learn, all_email))
        train_features, test_features, train_labels, test_labels = train_test_split(features, all_labels, test_size = 0.25, random_state = 42)
        self.__rf = RandomForestRegressor(n_estimators = 20)
        #with random
        #self.__rf = RandomForestRegressor(n_estimators = 20, random_state = 42)
        self.__rf.fit(train_features, train_labels);
    
    def prepare_data_learn(self, email):
        features = []
        features.append(email['emetteur'] == 'blabla@gmail.com')
        features.append(email['destinataire'] == self.__mail_box)
        features.append('notification' in email['objet'])
        return features

In [78]:

def test_classify():
    email_class = Email_classifier('d@gmail.com')
    boiteA = []
    boiteB = []
    faux_objet_email = {}

    faux_objet_email['emetteur'] = 'notif@sncf.com'
    faux_objet_email['destinataire'] = 'd@gmail.com'
    faux_objet_email['objet'] = 'notification'
    boiteA.append(faux_objet_email)

    faux_objet_email1 = {}
    faux_objet_email1['emetteur'] = 'test@gmail.com'
    faux_objet_email1['destinataire'] = 'd@gmail.com'
    faux_objet_email1['objet'] = 'notification'
    boiteA.append(faux_objet_email1)

    faux_objet_email2 = {}
    faux_objet_email2['emetteur'] = 'notification@gmail.com'
    faux_objet_email2['destinataire'] = 'd@gmail.com'
    faux_objet_email2['objet'] = 'notification'
    boiteA.append(faux_objet_email2)

    faux_objet_email3 = {}
    faux_objet_email3['emetteur'] = 'blabla@gmail.com'
    faux_objet_email3['destinataire'] = 'd@gmail.com'
    faux_objet_email3['objet'] = 'blablabla chaussure'
    boiteB.append(faux_objet_email3)

    faux_objet_email4 = {}
    faux_objet_email4['emetteur'] = 'blabla@gmail.com'
    faux_objet_email4['destinataire'] = 'd@gmail.com'
    faux_objet_email4['objet'] = 'blablabla tong'
    boiteB.append(faux_objet_email4)

    faux_objet_email5 = {}
    faux_objet_email5['emetteur'] = 'blabla@gmail.com'
    faux_objet_email5['destinataire'] = 'd@gmail.com'
    faux_objet_email5['objet'] = 'telephone'
    boiteB.append(faux_objet_email5)

    email_class.learn({'boite A': boiteA, 'boite B': boiteB})
    assert email_class.classify(faux_objet_email) == 'boite A'
    #print(email_class.classify(faux_objet_email))
    assert email_class.classify(faux_objet_email5) == 'boite B'
    #print(email_class.classify(faux_objet_email5))

test_classify()

[[1 0 0 2 2 1 2 0 0 1 1 0 2 2 2 0 0 0 0 2]]
[[2 0 0 1 1 2 1 0 0 2 2 0 1 1 1 0 0 0 0 1]]
